In [1]:
import pandas as pd
import re
import numpy as np
from konlpy import jvm
from konlpy.tag import Okt
import datetime

In [2]:
df = pd.read_csv('test.csv')

In [3]:
pattern = re.compile(r"\d+")
pattern_except_space = re.compile(r'[^\s]+')

In [4]:
for idx in df.columns:
    df[idx]=df[idx].replace("무관","-")

## 정규식 사용, 첫번째 숫자 age start, 두번째 end
df['start_age'] = df['연령'].apply(lambda x : pattern.findall(x)[0] if len(pattern.findall(x))>0 else " ".join(pattern_except_space.findall(x)))
df['end_age'] = df['연령'].apply(lambda x : pattern.findall(x)[1] if len(pattern.findall(x))>1 else " ".join(pattern_except_space.findall(x)))

## 이상, 이하 붙은것 처리
df['end_age']=df['end_age'].apply(lambda x : "-" if "이상" in x else x)
df['check less'] = df['end_age'].apply(lambda x : True if "이하" in x or "미만" in x else False)


df.loc[df['check less']==True , 'end_age'] = df[df['check less']==True]['start_age']
df.loc[df['check less']==True , 'start_age'] = "-"

In [5]:
df['age_except_less']=df['start_age'].apply(lambda x : True if len(pattern.findall(x))<1 else False )
df['age_except_more'] = df['start_age'].apply(lambda x : True if len(x) >2 else False)

In [6]:
df['age except'] = df['연령'].apply(lambda x : True if "(" in x or "경우" in x else False)

In [7]:
NON=['제한 없음','제한 없음','전 연령']
df.loc[df.start_age.isin(NON),'start_age'] = '-'
df.loc[df.end_age.isin(NON),'end_age'] = '-'
age_except_list = ['1988','재직 경험이 없는 청년']
df.loc[df['start_age'].isin(age_except_list),'age except'] = True

In [8]:
df['hash_tag'] = df['hash tag'].apply(lambda x : x.replace(',',""))

In [9]:
pattern_space = re.compile(r"[ㄱ-ㅎ가-힣0-9()]+")
pattern_univ = re.compile(r"(대학[^원생])|(대학$)")
pattern_haksa = re.compile(r"(학사)")
pattern_high = re.compile(r"(고교)|(고등학교)")
pattern_master = re.compile(r"(대학원)|(대학\(원\)|(석.{0,5}?사))|(^석$)")
pattern_doc = re.compile(r"(박.{0,3}사)")
pattern_grad = re.compile(r'(졸업[^예])|(졸업$)')
pattern_attend = re.compile(r'(재학)|(과정)')
pattern_null = re.compile(r'(무관)|(제한 ?없음)|(-$)')
p_more = re.compile(r'이상')
p_less = re.compile(r'이하')
p_excess = re.compile(r'초과')
p_under = re.compile(r'미만')
pattern_after_high = re.compile(r'(고졸)')
pattern_after_univ = re.compile(r'(대졸)')
pattern_attend_high = re.compile(r'고등학생')
pattern_attend_univ = re.compile(r'(대학생)|(대생$)')
pattern_expected =  re.compile(r'(예정$)')
pattern_grad_expected = re.compile(r'(졸업 ?예정.?)|(졸업\(예정.?\))')

In [10]:
def check_free(x):
    res_void = []
    if pattern_null.search(x):
        return res_void
    return pattern_space.findall(x)

In [11]:
def check_school_in_list(x):
    res = []
    
    for i in x:
        point = [] 
        if pattern_high.search(i):
            point.append('1 고등학교')
        if pattern_univ.search(i):
            point.append('1 대학교')
        if pattern_master.search(i):
            point.append('1 석사')
        if pattern_doc.search(i):
            point.append('1 박사')
        if pattern_haksa.search(i):
            point.append('1 학사')
        
        if pattern_attend.search(i):
            point.append('2 재학')
        if pattern_grad.search(i):
            point.append('2 졸업')
            
        ## 졸업 예정자 == eg 
        ## 고졸 ah
        ## 대졸 auN
        
        if pattern_grad_expected.search(i):
            point.append('23 졸업 예정')
            
        if pattern_after_high.search(i):
            point.append('12 고등학교 졸업')
            
        if pattern_after_univ.search(i):
            point.append('12 대학교 졸업')
            
        
        if pattern_attend_high.search(i):
            point.append('12 고등학교 재학')
        
        if pattern_attend_univ.search(i):
            point.append('12 대학교 재학')
            
        
        ## 이상 up
        ## 이하 dn
        ## 미만 ud
        ## 초과 ex
        ## 고졸 미만 , 대졸 이상 --> 이상은 없애도 될까? 
        ## 예정!!!!!!!!!!!!!!!!  ep
        
        if p_more.search(i):
            point.append('3 이상')
            
        if p_less.search(i):
            point.append('3 이하')
                
        if p_excess.search(i):
            point.append('3 초과')
            
        if p_under.search(i):
            point.append('3 미만')
            
        if pattern_expected.search(i):
            point.append('3 예정')
        
        res.append(point)
    return res
    

In [12]:
df['학력 파싱'] = df['학력'].apply(lambda x : check_free(x))
df['parse school']=df['학력 파싱'].apply(lambda x: check_school_in_list(x))

In [13]:
def list_destruct(x):
    res = []
    for idx in x:
        res.extend(idx)
    
    return res
        

In [14]:
df['parse list'] = df['parse school'].apply(lambda x : list_destruct(x)) 

In [15]:
def read_parsing_string(x):
    place = []
    condition = []
    plus_alpha = []
    res = []
    
    flag = 0    
    err=0
    for idx in x:
        
        ## 고등학교 대학교 석사 박사 1
        ## 졸업, 재학 2      고등학교 졸업은 12, 
        ## 석사, 박사는 뒤에 공백이면 재학 붙게
        ## 이상 미만 등 3
        
        proc = idx.split(' ')
        div = proc[0]
        if div =='1':
            #print('1')
            if flag!=1 and flag!=0:
                res.extend(list_multiply(place,condition,plus_alpha))
                place = []
                condition = []
                plus_alpha = []
                
            place.extend(proc[1:])
            flag=1
            
        elif div =='2':
            #print('2')
            if flag!=1 and flag!=2:
                #print('err in '+idx)
                err=1
            else:
                condition.extend(proc[1:])
                flag=2
          
        elif div =='3':
            #print('3')
            if flag == 0:
                #print('err in ' + idx)
                err=1
            else:
                plus_alpha.extend(proc[1:])
                flag=3
            

        elif div == '12':

            if flag!=1 and flag!=0:
                res.extend(list_multiply(place,condition,plus_alpha))
                place = []
                condition = []
                plus_alpha = []
            
            place.append(proc[1])
            condition.append(proc[2])    
            flag=2
        
        elif div =='23':
            if flag!=1 and flag!=0:
                condition.append(proc[1]+" "+proc[2])
                #plus_alpha.append()
                flag=3
            else:
                #print('err in ' + idx)
                err=1
        
    if '석사' in place or '박사' in place or '학사' in place:
        
        if len(condition)==0 and len(plus_alpha)==0:
            #print(place)
            condition.append('졸업')
    
    if err!=1:
        res.extend(list_multiply(place,condition,plus_alpha))
    
    return res

In [16]:
def list_multiply(x,y,z):
    res = []
    
    if len(z)>0:
        #print(x)
        #print(y)
        #print(z)
        if len(y)==0:
            res = [x[0]+" "+z[0]]
        else:
            res = [x[0]+" "+y[0]+" "+z[0]]
    else:
        for idx in x:
            for kda in y:
                res.append(str(idx)+" "+str(kda))
    
    return res

In [17]:
df['parse school res'] = df['parse list'].apply(lambda x: read_parsing_string(x)) 

In [18]:
okt = Okt()

C:\Users\moggi\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [19]:
def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

In [20]:
pattern_front_dash = re.compile(r".*(?=\/)")
pattern_ggage = re.compile(r"까지")
pattern_butu = re.compile(r"부터")
pattern_menun = re.compile(r'매년')
pattern_mewal = re.compile(r'매월')
pattern_me = re.compile(r'^매\/')
pattern_d_wave = re.compile(r'[~∼]')
pattern_cha = re.compile(r"^차\/")
pattern_yeon = re.compile(r"^연\/")
pattern_front_Punctuation = re.compile(r".*(?=\/Punctuation)")
pattern_only_num = re.compile(r"[0-9]+")

In [21]:
def token_num(x):
    
    res = []
    Punctuation_list = []
    for idx in x:
        if "Number" in idx:
            
            ## 2019.12 
            if "." in idx:
                dot_process = pattern_front_dash.search(idx).group()
                dot_list=pattern_only_num.findall(dot_process)
                res.append(dot_list[0])
                res.append(".")
                res.append(dot_list[1])
            elif "-" in idx:
                dot_process = pattern_front_dash.search(idx).group()
                dot_list=pattern_only_num.findall(dot_process)
                res.append(dot_list[0])
                res.append(".")
                res.append(dot_list[1])
            
            elif pattern_d_wave.search(idx):
                wave_process = pattern_front_dash.search(idx).group()
                wave_list=pattern_only_num.findall(wave_process)
                res.append(wave_list[0])
                res.append("~")
                res.append(wave_list[1])

            else:
                res.append(pattern_front_dash.search(idx).group())
        elif "까지" in idx:
            
            index_ggage = x.index(idx)
            if "Number" in x[index_ggage-1] or "Number" in x[index_ggage-2]:
                res.append("까지")
                
        elif "부터" in idx:
            res.append("부터")
        elif "매년" in idx:
            res.append("매년")
        elif "매월" in idx:
            res.append("매월")
        elif "학기" in idx:
            res.append("학기")
        elif pattern_me.search(idx):
            res.append("매")
        elif "상시" in idx:
            res.append("상시")
        elif pattern_d_wave.search(idx):
            res.append("~")
        elif pattern_cha.search(idx):
            res.append("차")
        elif pattern_yeon.search(idx):
            res.append("연")
        elif "/Punctuation" in idx:
            res.append(pattern_front_Punctuation.search(idx).group())
            if pattern_front_Punctuation.search(idx).group() not in Punctuation_list:
                Punctuation_list.append(pattern_front_Punctuation.search(idx).group())
                
    
    return res

In [22]:
df['token'] = df['신청기간'].apply(lambda x:tokenize(str(x)));

In [23]:
df['num_token'] = df['token'].apply(lambda x : token_num(x))

In [27]:
pattern_number = re.compile(r"[0-9\.]+")
pattern_not_number =re.compile(r"[^0-9\.]+")
pattern_year = re.compile(r"년$")
pattern_month = re.compile(r"월$")
pattern_day = re.compile(r"일$")
pattern_only_num = re.compile(r"[0-9]+")
pattern_cha = re.compile(r"^차\/")
pattern_yeon = re.compile(r"^연\/")
pattern_hoe = re.compile(r"회")
pattern_number_only = re.compile("[0-9]+")

In [28]:
def process_time(x):
    res=[]
    process_list = []
    flag = 0
    max_flag = 0
    waved = 0
    every_year = 0
    every_month = 0
    
    first_num_flag=0
    
    for idx in x:
        
        if every_year == 1 or "매년" in idx:
            if flag <1:
                flag =1
            every_year=1
        
        if every_month ==1 or "매월" in idx:
            if flag < 2:
                flag = 2
            every_month = 1
        
        ## 일단 차, 학기 앞 num 제거 19.1학기
        ## 수정필
        if idx in ["차","학기"]: 
            #if "학기" in idx:
            #    res.append(".".join(process_list) + "학기")
            #else:
            #    res.append(".".join(process_list) + "차")
            ## 년만 제거 할게요 
            if flag ==1 or flag == 0:
                process_list = []
                
            else :
                res.append(".".join(process_list[:-1]))
                process_list = []
            
            flag = 0
            
        if ":" in idx or pattern_hoe.search(idx):
            continue
        
        #print("idx = ")
        #print(idx)
        #print("flag=")
        #print(flag)
        #print(len(res))
        #print("max_flag = ")
        #print(max_flag)
        #print(res)
        
        if flag == 3 or (first_num_flag==1 and flag==max_flag):
            
            if first_num_flag==0:
                max_flag = flag
                first_num_flag=1
                
            res.append(".".join(process_list))
                        
            
            flag=0
            process_list = []
        ##print(x)
        ##print(idx)
        if pattern_number_only.search(idx):
            ## 13 년도 이상
            if (flag == 0 and int(pattern_number_only.search(idx).group())>12 and not pattern_day.search(idx)) or pattern_year.search(idx):
                process_list.append(str(pattern_number_only.search(idx).group()) + "년")
                flag=1
            elif flag==1 or pattern_month.search(idx) or (flag==0 and int(pattern_number_only.search(idx).group())<13):
                
                ## 1월 / 2 월
                if flag == 2:
                    if first_num_flag==0:
                        max_flag = flag
                        first_num_flag=1
                    res.append(".".join(process_list))
                    process_list = []
                    
                process_list.append(str(pattern_number_only.search(idx).group())+"월")
                flag=2
                
                
            elif flag==2 or pattern_day.search(idx):
                process_list.append(str(pattern_number_only.search(idx).group())+"일")
                flag=3
            ##print(flag)
            ##print(res)
        else:
            if idx =="~":
                if len(process_list)>0:
                    res.append(".".join(process_list))
                    process_list = []
                if first_num_flag==0:
                    max_flag = flag
                    first_num_flag=1
                    
                waved = 1
                res.append("~")
                flag=0
                
            if idx in ["까지","부터","매년","매월"]:
                
                if len(res)==0 and idx in ["까지","부터"]:
                    max_flag = flag
                
                    res.append(".".join(process_list))
                    flag=0
                    process_list = []
                
                res.append(idx)
            
            #if "회" in idx:
            #    res.append(idx)
            
    if flag>=1 and len(process_list)>0:
        res.append(".".join(process_list))
    
    return res

In [29]:
df['process_timed']=df['num_token'].apply(lambda x : process_time(x))

In [30]:
pattern_numyear = re.compile("[0-9]*(?=년)")
pattern_nummonth = re.compile("[0-9]*(?=월)")
pattern_numday = re.compile("[0-9]*(?=일)")

In [31]:
def process_time_to_date_time(x):
    
    res_start = []
    res_end = []
    
    
    ## start, end list에 번갈아가면서 입력
    start_or_end_date=1
    
    for idx in x:
        ## 하나라도 날짜가 매칭되면 res에 넣는다.
        input_flag = 0
        year_flag = 0
        month_flag = 0
        day_flag =0
        
        
        year = datetime.datetime.now().year ## 추후 크롤링 일자로 대체
        
        ## TODO '매월' 처리 ????
        if idx =="매월" or idx == "매년":
            continue
        
        if idx == "까지" and len(res_end)==0:
            res_end = res_start
            res_start=[]
        
        month = datetime.datetime.now().month 
        
        ## day? start에는 1 end에는 30
        day = -1
        
        if pattern_numyear.search(idx):
            year=int(pattern_numyear.search(idx).group())
            if int(year/100)==0:
                year = year + 2000
            
            year_flag = 1
            input_flag=1
        
        if pattern_nummonth.search(idx):
            month = int(pattern_nummonth.search(idx).group())
            month_flag = 1
            input_flag=1
        
        if pattern_numday.search(idx):
         
            day = int(pattern_numday.search(idx).group())
            input_flag=1
            
        if input_flag==1:
            ##print(x)
            ##print(month)
            if start_or_end_date ==1:
                if day == -1:
                    day = 1
               
                res_start.append(datetime.datetime(year = year, month = month,day =day))
                start_or_end_date=2
                
            elif start_or_end_date==2:
                if day == -1:
                    day = 28
                res_end.append(datetime.datetime(year = year, month = month,day =day))
                start_or_end_date=1
                
    return res_start,res_end

In [41]:
def for_test_db_date(x):
    if len(x)>0:
        return x[0]
    else:
        return None

In [42]:
df['start_date']= df['process_timed'].apply(lambda x: for_test_db_date(process_time_to_date_time(x)[0]))

In [44]:
df['end_date']= df['process_timed'].apply(lambda x: for_test_db_date(process_time_to_date_time(x)[1]))

In [45]:
df.columns

Index(['Unnamed: 0', 'id', 'title', 'hash tag', '설명', '연령', '학력', '전공', '취업상태',
       '특화분야', '기업', '신청기간', '기타내용', 'start_age', 'end_age', 'check less',
       'age_except_less', 'age_except_more', 'age except', 'hash_tag', '학력 파싱',
       'parse school', 'parse list', 'parse school res', 'token', 'num_token',
       'process_timed', 'start_date', 'end_date', 'url'],
      dtype='object')

In [46]:
url_id_base = "https://www.youthcenter.go.kr/jynEmpSptNew/jynEmpSptGuide.do?bizId="

In [47]:
df['url']=df['id'].apply(lambda x: url_id_base+str(x))

In [48]:
df.columns

Index(['Unnamed: 0', 'id', 'title', 'hash tag', '설명', '연령', '학력', '전공', '취업상태',
       '특화분야', '기업', '신청기간', '기타내용', 'start_age', 'end_age', 'check less',
       'age_except_less', 'age_except_more', 'age except', 'hash_tag', '학력 파싱',
       'parse school', 'parse list', 'parse school res', 'token', 'num_token',
       'process_timed', 'start_date', 'end_date', 'url'],
      dtype='object')

In [50]:
df['start_date']

0            NaT
1            NaT
2     2019-10-01
3            NaT
4            NaT
5     2018-07-01
6     2018-07-31
7            NaT
8            NaT
9            NaT
10    2019-01-01
11           NaT
12           NaT
13           NaT
14    2019-01-01
15           NaT
16    2019-10-01
17           NaT
18           NaT
19           NaT
20           NaT
21    2019-09-01
22           NaT
23           NaT
24           NaT
25           NaT
26           NaT
27    2019-01-22
28           NaT
29           NaT
         ...    
111          NaT
112          NaT
113          NaT
114          NaT
115          NaT
116          NaT
117          NaT
118          NaT
119          NaT
120          NaT
121          NaT
122          NaT
123          NaT
124          NaT
125   2018-07-01
126          NaT
127          NaT
128   2019-11-01
129          NaT
130          NaT
131   2019-02-19
132          NaT
133   2018-10-01
134          NaT
135          NaT
136   2019-05-01
137          NaT
138   2018-05-

In [51]:
df[['title','url','설명','hash tag','start_age','end_age','start_date','end_date','parse school res']].to_csv("temp_db2.csv",encoding="utf-8-sig")